In [52]:
!pip install yahooquery

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 54, Finished, Available, Finished)

In [53]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr, to_date, month, year, when, lag, avg
from pyspark.sql.window import Window
import pandas as pd
import numpy as np

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 55, Finished, Available, Finished)

In [54]:
from yahooquery import Ticker

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 56, Finished, Available, Finished)

In [55]:
from pyspark.sql.functions import round

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 57, Finished, Available, Finished)

In [56]:
#Carregar tabela CSV
df_tickers = spark.read.option("header", True).csv('Files/acoes_listadas_b3.csv')
display(df_tickers)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 58, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 141a45c2-b49c-4bd5-8f1a-2678c5c8efe2)

In [57]:
#Selecionar itens da linhas da 1ª coluna com 3
df_tickers = df_tickers.filter(col('Ticker').endswith('3'))
display(df_tickers)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 59, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, ac3b9a3d-0522-4ed6-804a-8158cf2d51c5)

In [58]:
#Concatenar itens das linhas da 1ª coluna com .SA
df_tickers = df_tickers.withColumn('Ticker', expr("concat(Ticker, '.SA')"))
display(df_tickers)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 60, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a3df82d4-abe5-4100-9c7b-9083540d81da)

In [59]:
lista_tickers = [row['Ticker'] for row in df_tickers.collect()]

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 61, Finished, Available, Finished)

In [60]:
lista_tickers_resumido = ['PETR3.SA', 'VALE3.SA', 'TAEE3.SA']

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 62, Finished, Available, Finished)

In [61]:
tickers = Ticker(lista_tickers)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 63, Finished, Available, Finished)

In [62]:
df_cotacoes = tickers.history(period='30d', adj_timezone=False)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 64, Finished, Available, Finished)

In [63]:
df_cotacoes = df_cotacoes.reset_index()
display(df_cotacoes)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 65, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 7a2e9c18-c422-4517-9807-68afa0c09e71)

In [64]:
#Conversão para DataHora
df_cotacoes['date'] = pd.to_datetime(df_cotacoes['date'], utc=True)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 66, Finished, Available, Finished)

In [65]:
#Fuso horário de São Paulo (-3:00)
df_cotacoes['date'] = df_cotacoes['date'].dt.tz_convert('America/Sao_Paulo')

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 67, Finished, Available, Finished)

In [66]:
#converter para Data (dd/mm/aaaa)
df_cotacoes['date'] = df_cotacoes['date'].dt.date

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 68, Finished, Available, Finished)

In [67]:
#Transformação do DataFrame Pandas em Spark
df_cotacoes_spark = spark.createDataFrame(df_cotacoes)

#Renomeação de Colunas
df_cotacoes_spark = df_cotacoes_spark.withColumnRenamed('date', 'Data') \
   .withColumnRenamed('open', 'Abertura') \
   .withColumnRenamed('high', 'Alta') \
   .withColumnRenamed('low', 'Baixa') \
   .withColumnRenamed('close', 'Fechamento') \
   .withColumnRenamed('volume', 'Volume') \
   .withColumnRenamed('adjclose', 'FechamentoAjustado') \
   .withColumnRenamed('symbol', 'Ticker') \
   .withColumnRenamed('dividends', 'Dividendos')

df_cotacoes_spark = df_cotacoes_spark \
    .withColumn('Abertura', round('Abertura', 2)) \
    .withColumn('Alta', round('Alta', 2)) \
    .withColumn('Baixa', round('Baixa', 2)) \
    .withColumn('Fechamento', round('Fechamento', 2)) \
    .withColumn('Volume', round('Volume', 2)) \
    .withColumn('FechamentoAjustado', round('FechamentoAjustado', 2)) \
    .withColumn('Dividendos', round('Dividendos', 2))

display(df_cotacoes_spark)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 69, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, a9225386-a54b-43f1-9817-eba74aec0846)

In [68]:
#Cálculo da Variação de Fechamento
df_cotacoes_spark = df_cotacoes_spark.sort(['Ticker', 'Data'])
display(df_cotacoes_spark)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 70, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 72aa2d2c-83aa-44fb-aefa-1e305069e607)

In [69]:
#Define janela por Ticker Ordenada por Data
windowSpec = Window.partitionBy('Ticker').orderBy('Data')

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 71, Finished, Available, Finished)

In [70]:
#Criação da coluna de 1 dia antes do fechamento
df_cotacoes_spark = df_cotacoes_spark.withColumn("Fechamento_Anterior", lag("Fechamento").over(windowSpec))
display(df_cotacoes_spark)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 72, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0020f1a5-5e90-4409-8d1a-f5abbd0e1949)

In [71]:
# Cria coluna com a variação percentual do fechamento em relação ao dia anterior
df_cotacoes_spark = df_cotacoes_spark.withColumn("Fechamento_Variacao", 
    (col("Fechamento") - col("Fechamento_Anterior")) / col("Fechamento_Anterior"))

df_cotacoes_spark = df_cotacoes_spark.withColumn(
    'Fechamento_Variacao',
    round(col('Fechamento_Variacao') * 100, 2)
)
display(df_cotacoes_spark)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 73, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 497c571a-9c55-4b14-ae19-5519d290060e)

In [72]:
# Converte o dicionário de perfil dos ativos (como setor, indústria etc.) em DataFrame pandas
df_setores = pd.DataFrame(tickers.asset_profile)

# Transpõe o DataFrame: linhas viram colunas e colunas viram linhas (tickers passam a ser índice)
df_setores = df_setores.T

display(df_setores)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 74, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 2c9551cd-f228-4270-a63f-d43bb2a35d1e)

In [73]:
# Seleciona apenas as colunas de setor e indústria e reseta o índice (ticker vira coluna)
df_setores = df_setores[['sectorDisp', 'industryDisp']].reset_index()
display(df_setores)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 75, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6edd2766-15a3-46de-9085-42905be3c76a)

In [74]:
# Renomeia a coluna 'index' para 'Ticker' (após o reset_index)
df_setores = df_setores.rename(columns = {'index': 'Ticker'})
display(df_setores)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 76, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 6ec75595-4eb2-4747-b424-a59d3986e427)

In [75]:
# Converte o DataFrame Pandas df_setores para um DataFrame Spark
df_setores_spark = spark.createDataFrame(df_setores)
display(df_setores_spark)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 77, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 5f0fe174-dc30-49c1-a1fb-446393df5cac)

In [76]:
# Junta os dados de cotações com os dados setoriais pelo Ticker, mantendo todas as cotações (left join)
df_cotacoes_spark = df_cotacoes_spark.join(df_setores_spark, on='Ticker', how='left')
display(df_cotacoes_spark)

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 78, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 99736fe5-1cdd-4009-ae07-2b05e28eb026)

In [77]:
# Salva o DataFrame como uma tabela Delta no metastore Spark com schema merge e sobrescrita
df_cotacoes_spark.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .saveAsTable("cotacoes_tickers")

StatementMeta(, 4867205c-1dc9-484c-8763-0d4a37416ea6, 79, Finished, Available, Finished)